In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import tushare as ts
import datetime
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA

In [16]:
data = pd.read_csv('./人口-预期寿命-GDP.csv')

In [17]:
data

,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710
3,Afghanistan,1967,11537966,Asia,34.020,836.197138
4,Afghanistan,1972,13079460,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563,Africa,39.989,672.038623


In [18]:
gdp_year_data = data.loc[::-1, ['year', 'gdp_cap']]

In [19]:
gdp_year_data

,year,gdp_cap
1703,2007,469.709298
1702,2002,672.038623
1701,1997,792.449960
1700,1992,693.420786
1699,1987,706.157306
...,...,...
4,1972,739.981106
3,1967,836.197138
2,1962,853.100710
1,1957,820.853030


In [20]:
gdp_year_data = gdp_year_data.set_index('year')

In [21]:
gdp_year_data

,gdp_cap
year,
2007,469.709298
2002,672.038623
1997,792.449960
1992,693.420786
1987,706.157306
...,...
1972,739.981106
1967,836.197138
1962,853.100710


In [22]:
gdp_series = gdp_year_data['gdp_cap']

In [23]:
gdp_series.head()

year
2007    469.709298
2002    672.038623
1997    792.449960
1992    693.420786
1987    706.157306
Name: gdp_cap, dtype: float64